In [1]:
from models import EfficientNet3D
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
batch_size = 8
inputs_shape = (batch_size, 45, 6, 6, 6)
labels_shape = (batch_size,)
inputs = torch.randn(inputs_shape).to(device)
labels = torch.randint(low=0, high=26, size=labels_shape).to(device)

cnn_flattened_size = inputs_shape[1] * inputs_shape[2] * inputs_shape[3] * inputs_shape[4]

print("inputs shape", inputs.shape)
print("labels shape", labels.shape)

inputs shape torch.Size([8, 45, 6, 6, 6])
labels shape torch.Size([8])


In [3]:
num_classes = 27
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': num_classes}, in_channels=1, cnn_flattened_size=cnn_flattened_size, hidden_size=128, task="classification")

In [4]:
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train()

EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[2, 2, 2], groups=32, bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticSamePadding(
        32, 24, kernel_siz

In [5]:
previous_prediction_1 = torch.randn((batch_size, num_classes))
previous_prediction_2 = torch.randn((batch_size, num_classes))
# Concatenate the previous predictions together along dimension 2
previous_predictions = torch.cat((previous_prediction_1, previous_prediction_2), dim=1).cuda()
print("Previous prediction size outside", previous_predictions.shape)

Previous prediction size outside torch.Size([8, 54])


In [7]:
for epoch in range(1):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs, previous_predictions)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')
print(outputs.shape)
print(outputs.cpu().detach().numpy())
print(outputs.cpu().detach().numpy().sum(axis=1))

[1] loss: 3.297
Finished Training
torch.Size([8, 27])
[[0.03757453 0.03140101 0.03295003 0.03433231 0.03660331 0.03231703
  0.03382427 0.03994634 0.02770792 0.02432857 0.04628998 0.03080159
  0.04892669 0.04645617 0.04502131 0.0386044  0.04154273 0.0419201
  0.02834877 0.02597728 0.02790319 0.04849401 0.03070141 0.04102479
  0.04996203 0.0381685  0.03887175]
 [0.03827504 0.03961461 0.03499173 0.04502687 0.03537461 0.02862286
  0.03232015 0.03155469 0.0330483  0.03927643 0.04324983 0.02947355
  0.03957387 0.03927169 0.03676104 0.03505629 0.04542359 0.03836203
  0.04444911 0.03322186 0.02846921 0.04317406 0.03411331 0.04091558
  0.03486805 0.03252089 0.04299076]
 [0.03298941 0.03412053 0.03608302 0.04291343 0.04077852 0.03537536
  0.0334267  0.02996182 0.04151939 0.02578353 0.0351865  0.02888839
  0.043412   0.04183607 0.03802723 0.03925906 0.03297287 0.03025452
  0.03563792 0.03367005 0.0382796  0.04225047 0.03207859 0.05824609
  0.03979136 0.03738441 0.03987323]
 [0.03344912 0.03347397